# Exploratory Data Analysis for the Delta Analytics Teaching Fellowship

**Author:** *Cynthia Thinwa*

## INTRODUCTION

### DATA HANDLING PRACTICES:

* Based on Twitter API best practice, the actual data will not be shared, only Twitter's tweet IDs for future reference
* The data will be cleaned to remove personally identifiable information like emails and phone numbers
* Content published on social media platforms belongs to the public domain with the various authors having given consent

Data collection was done from a Microsoft command line running

`twint -s %23kot --since "2020-06-01 03:00:00" --until "2021-06-01 03:00:00" -o kotdata.csv --csv`

## EXPLORATORY DATA ANALYSIS

### Introduction

The raw data was loaded as follows, with the following characteristics:


In [ ]:
import numpy as np
import pandas as pd
import os
import json

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

library(dplyr)
library(wordcloud)
library(RColorBrewer)
library(rtweet)
library(tidytext)
library(ggplot2)
library(wordcloud2)

1. The number of tweets:

In [ ]:
%%R

url <- "C:/storage/Personal drive backup/Career/Post-Masters/Delta Analytics Teaching Fellowship/EDA/2.kotdata.csv"
DATFdata <- read.delim(url)
dim(DATFdata)[1]


2. The number of unique conversations had:


In [ ]:
%%R

DATFdata$conversation_id <- factor(DATFdata$conversation_id)
DATFdata$id <- factor(DATFdata$id)

dim(as.data.frame(table(DATFdata$conversation_id)))[1]


3. The number of unique users speaking:


In [ ]:
%%R

DATFdata$user_id <- factor(DATFdata$user_id)

dim(as.data.frame(table(DATFdata$user_id)))[1]


4. The most frequent language of posting:


In [ ]:
%%R

lang <- as.data.frame(table(DATFdata$language))
colnames(lang) <- c('Language','Frequency')
head(lang[order(lang$Freq, decreasing = TRUE),],n=1)

In [ ]:
%%R

DATFdata$tweet[1:20]


5. The date on which most tweets were posted (tweets were from 1st June 2020 UTC+3 upto 1st June 2021 UTC+3): 


In [ ]:
%%R

dates <- as.data.frame(table(DATFdata$date))
colnames(dates) <- c('Date','Frequency')
head(dates[order(dates$Freq, decreasing = TRUE),],n=1)


### Text transformation

Text cleaning was as follows, using `eng_tweets$tweet[4]` as an example:


In [ ]:
%%R

# Get organic tweets first; found that all tweets were organic!

# get only English ones:
eng_tweets <- DATFdata[DATFdata$language=='en',]; print(eng_tweets$tweet[4])

# Remove funny symbols
eng_tweets$tweet <- iconv(eng_tweets$tweet, from = 'UTF-8', to = 'ISO-8859-1', sub = ''); print(eng_tweets$tweet[4])

eng_tweets$tweet <- iconv(eng_tweets$tweet, from = 'ISO-8859-1', to = 'UTF-8', sub = ''); print(eng_tweets$tweet[4])

eng_tweets$tweet <- gsub("https\\S*", "", eng_tweets$tweet); print(eng_tweets$tweet[4]) #remove urls

eng_tweets$tweet <- gsub("@", "", eng_tweets$tweet); print(eng_tweets$tweet[4]) #remove mentions symbol

eng_tweets$tweet <- gsub("#*", "", eng_tweets$tweet); print(eng_tweets$tweet[4]) #remove hashtags symbol

eng_tweets$tweet <- gsub("[\r\n]", " ", eng_tweets$tweet); print(eng_tweets$tweet[4]) #remove newline characters

#(we have separate columns with the details)
# Punctuation was managed as follows:
eng_tweets$tweet <- gsub("'", "", eng_tweets$tweet); print(eng_tweets$tweet[4])

eng_tweets$tweet <- gsub("[[:punct:]]", " ", eng_tweets$tweet); print(eng_tweets$tweet[4])

eng_tweets$tweet <- gsub("amp", "", eng_tweets$tweet); print(eng_tweets$tweet[4]) # remove ampersands

# Finally, everything was made lowercase
eng_tweets$tweet <- tolower(eng_tweets$tweet); print(eng_tweets$tweet[4])

In [ ]:
%%R

# Tokenize words
Words <- eng_tweets %>%
  select(tweet) %>%
  unnest_tokens(word, tweet)


### Word Frequency


In [ ]:
%%R

Words %>% # gives you a bar chart of the most frequent words found in the tweets
  count(word, sort = TRUE) %>%
  top_n(15) %>%
  mutate(word = reorder(word, n)) %>%
  ggplot(aes(x = word, y = n)) +
  geom_col() +
  xlab(NULL) +
  coord_flip() +
  labs(y = "Count",
       x = "Unique words",
       title = "Most frequent words found in the #KOT tweets")

In [ ]:
%%R

# remove stop words
Words <- Words %>%
  anti_join(stop_words)

Words %>% # gives you a bar chart of the most frequent words found in the tweets
  count(word, sort = TRUE) %>%
  top_n(15) %>%
  mutate(word = reorder(word, n)) %>%
  ggplot(aes(x = word, y = n)) +
  geom_col() +
  xlab(NULL) +
  coord_flip() +
  labs(y = "Count",
       x = "Unique words",
       title = "Most frequent words found in the #KOT tweets",
       subtitle = "Stop words removed from the list")


### Hashtag Frequency

All tweets have the hashtag `#KOT`, but we are interested in what else this community talks about, so this hashtag will have to be removed from our visualization.


In [ ]:
%%R

eng_tweets$hashtags <- as.character(eng_tweets$hashtags)

eng_tweets$hashtags <- gsub("[[:punct:]]", "", eng_tweets$hashtags)

# every hashtag was made lowercase
eng_tweets$hashtags <- tolower(eng_tweets$hashtags)

# and #KOT was removed before tokenization
eng_tweets$hashtags <- gsub("kot", "", eng_tweets$hashtags)

Hashtags <- eng_tweets %>%
  select(hashtags) %>%
  unnest_tokens(word, hashtags)

Hashtags_count <- as.data.frame(table(Hashtags$word))

viz1 <- wordcloud2(Hashtags_count, size=0.7)

![](viz1.PNG)

In [ ]:
%%R

head(Hashtags_count[order(Hashtags_count$Freq, decreasing = TRUE),], n=10)

Based on the results above

* platform (`#loyals`),
* patriotism (`#kenya` and `#nairobi`),
* radio (`#mainaandkingangi` and `#teamclassic`),
* political trends (`#bbinonsense` and `#punguzamizigo`)
* job ads(`#ikokazike` and `#ikokazi`) and
* cats (`#cat` and `#gato`)

took centre stage. 


### People of Influence

In this area, we are interested in identifying people mentioned most frequently.

The text, using `eng_tweets$mentions[4]` as an example was first converted as follows:


In [ ]:
%%R

eng_tweets$mentions[4]
# Remove funny symbols
eng_tweets$mentions <- iconv(eng_tweets$mentions, from = 'UTF-8', to = 'UTF-8');eng_tweets$mentions[4]

eng_tweets$mentions <- stringr::str_replace_all(
  eng_tweets$mentions, "\'", "\"")
eng_tweets$mentions <- gsub("[", "", eng_tweets$mentions, fixed = TRUE)
eng_tweets$mentions <- gsub("]", "", eng_tweets$mentions, fixed = TRUE)

df <- as.data.frame(eng_tweets$mentions); dim(df)
df[df == ""] <- NA  
df <- na.omit(df); dim(df)


In [ ]:
#get file location
os.getcwd()

In [ ]:
%%R

write.csv(df$`eng_tweets$mentions`,'C:/Users/CT/Documents/GitHub/Delta-Analytics-2021-CT-Project/dicts.csv')

In [ ]:
dicts = pd.read_csv('dicts.csv')
dicts.head(5)

In [ ]:
# Delete the column
dicts = dicts.drop(labels='Unnamed: 0', axis=1)
dicts.shape

In [ ]:
dicts.head(5)

In [ ]:
single_dicts_copy = dicts.copy()
single_dicts_copy.iloc[:,0] = single_dicts_copy.iloc[:,0].str.replace('{','',regex=False)
single_dicts_copy.iloc[:,0] = single_dicts_copy.iloc[:,0].str.replace('}','',regex=False)
single_dicts_copy.iloc[:,0] = single_dicts_copy.iloc[:,0].str.replace(' ','',regex=False)
single_dicts_copy.iloc[:,0] = single_dicts_copy.iloc[:,0].str.replace("\"",'',regex=False)
single_dicts_copy.columns = ['dictionary']
print(single_dicts_copy.iloc[0,0],'\n')
print(single_dicts_copy.iloc[1,0])

In [ ]:
long_lst = list(single_dicts_copy.iloc[:,0].values)
info = []
i=0

for i in range(len(single_dicts_copy.iloc[:,0])):
    item = long_lst[i].split(",")
    info.append(item)

print(len(info),'\n')
info[0]

In [ ]:
screen_names = []
i=0
# initializing substring
subs = 'screen_name'

for i in range(len(info)):
    item = list(filter(lambda x: subs in x, info[i]))
    screen_names.append(item)

print(len(screen_names),'\n')
screen_names[0:5]

In [ ]:
ids = []
i=0
# initializing substring
subs = 'id:'

for i in range(len(info)):
    item = list(filter(lambda x: subs in x, info[i]))
    ids.append(item)

print(len(ids),'\n')
ids[0:5]

In [ ]:
flat_list = [item for sublist in screen_names for item in sublist]
screen_names_s = pd.Series(flat_list)
screen_names_s.shape

In [ ]:
flat_list = [item for sublist in ids for item in sublist]
ids_s = pd.Series(flat_list)
ids_s.shape

In [ ]:
data = {"id": ids_s,
        "screen_name": screen_names_s}

persons_df = pd.concat(data,axis=1)
persons_df.head(5)

In [ ]:
# Remove identifiers
persons_df['id'] = persons_df['id'].str.replace('id:','',regex=False)
persons_df['screen_name'] = persons_df['screen_name'].str.replace('screen_name:','',regex=False)
persons_df.head(5)

In [ ]:
# Export to R for wordcloud
persons_df.to_csv('C:/Users/CT/Documents/GitHub/Delta-Analytics-2021-CT-Project/Tweeps.csv')

In [ ]:
%%R

persons <- read.csv('C:/Users/CT/Documents/GitHub/Delta-Analytics-2021-CT-Project/Tweeps.csv')

Tweep_count <- as.data.frame(table(persons$screen_name))

viz2 <- wordcloud2(Tweep_count, size=0.7)

![](viz2.PNG)

In [ ]:
%%R

head(Tweep_count[order(Tweep_count$Freq, decreasing = TRUE),],n=10)

Based on the result above, the most mentioned 'persons' over the past year on Twitter were 

* political institutions such as `@statehousekenya`, `@dcikenya` and `@nassemblyke`,
* a betting firm, `@safebetske`,
* politicians such as `@railaodinga` (the leader of Kenyan opposition parties) and `@williamsruto` (the current deputy vice president) as well as
* media houses such as `@citizentvkenya`, `@ntvkenya` and `@classic105kenya` and
* an inspiration account, `@dodzweit` (A reverend who posts inspirational and at times Christian content)

A possible explanation for this phenomenon is that 2022 will be an election year, and the two political leaders are running against each other and campaigning online.

However,

* `@dodzweit` was mentioned only by the church s/he pastors, `@cotchurchhq`, primarily in short summaries of his/her talks.
* `@safebetske` was mentioned only by a forex trader, `@theforexguyke`, primarily in retweets.

## MODELLING

In [ ]:
%%R

model_data <- cbind.data.frame(eng_tweets$id, eng_tweets$conversation_id,
                               eng_tweets$date, eng_tweets$time,
                               eng_tweets$user_id, eng_tweets$tweet,
                               eng_tweets$mentions, eng_tweets$hashtags)
write.csv(model_data, 'eng_model_data.csv')
model_data[1:5,]

In [ ]:
data = pd.read_csv('eng_model_data.csv')
data

In [ ]:
model_data = data.copy()

print(model_data.dtypes)

model_data['eng_tweets$id'] = model_data['eng_tweets$id'].astype('category')
model_data['eng_tweets$conversation_id'] = model_data['eng_tweets$conversation_id'].astype('category')
model_data['eng_tweets$user_id'] = model_data['eng_tweets$user_id'].astype('category')
model_data['timestamp'] = model_data['eng_tweets$date'] + ' ' + model_data['eng_tweets$time']
model_data['timestamp'] = pd.to_datetime(model_data['timestamp'])
model_data = model_data.drop(['Unnamed: 0', 'eng_tweets$date', 'eng_tweets$time',
                              'eng_tweets$mentions', 'eng_tweets$hashtags'],1)

print('\n', model_data.dtypes)

In [ ]:
model_data.columns = ['tweet_id', 'conversation_id', 'user_id', 'tweet', 'timestamp']
model_data

In [ ]:
#!pip install gensim
#!pip install python-Levenshtein
from gensim.models.doc2vec import Doc2Vec, TaggedDocument #tokenize and tag each tweet
model_data['tagged_tweets'] = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(model_data.tweet)]#display the tagged docs

#model_data = model_data.drop(['tweet'],1) #drop the raw tweets
model_data

Based on the data above, unsupervised machine learning will be applied to:
* tranform tweets into numerical vectors (learn more [here](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)) using [Doc2Vec](https://towardsdatascience.com/how-to-vectorize-text-in-dataframes-for-nlp-tasks-3-simple-techniques-82925a5600db)
* use k-means to [classify the tweets](https://towardsdatascience.com/unsupervised-sentiment-analysis-a38bf1906483), constructing a label in the process.
* use Catboost to [predict the constructed label](https://towardsdatascience.com/unconventional-sentiment-analysis-bert-vs-catboost-90645f2437a9)

In [ ]:
# transform the vectors into numerical vectors
model = Doc2Vec(dm=1, vector_size=32, min_count=1, workers=8, epochs = 20) #instantiate model
model.build_vocab(model_data['tagged_tweets']) #build vocab
model.train(model_data['tagged_tweets'], total_examples=model.corpus_count,
            epochs=model.epochs) # train the vectorization model

#generate vectors
tweet_vec = [model.infer_vector((model_data['tweet'][i].split(' '))) 
            for i in range(0,len(model_data['tweet']))]
tweet_vec[0]

In [ ]:
tweet_vector = np.array(tweet_vec).tolist() #Create a list of lists
model_data['tweet_vector'] = tweet_vector #set list to dataframe column
model_data

In [ ]:
#!pip install sklearn
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(tweet_vec) # classify data into 2 pools based on document vectors
model_data['label'] = pd.Series(kmeans.labels_)

model_data

In [ ]:
# Tweets labeled 0:
i=0
for i in range(26):
    if model_data['label'][i] == 0:
        print(model_data.tweet[i], '\n', '\n')
    i+=1

In [ ]:
# Tweets labeled 1:
i=0
for i in range(26):
    if model_data['label'][i] == 1:
        print(model_data.tweet[i], '\n', '\n')
    i+=1

The results above indicate that there is misuse of hashtags to push products and there is no clear positive sentiment or negative sentiment in any of the classes.